In [18]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from google.cloud import bigquery
from google.cloud import storage
from google.oauth2 import service_account

# Ruta al archivo JSON de las credenciales
key_path = '/Users/negro/Library/CloudStorage/OneDrive-Personal/Documentos/00 Fran/01 - Personales/02-Learn/0. Data Science/0. Data Science/2_projects/1_final/webScrap/GCP/nyc-taxis-and-carbon-emission-195b3347e44f.json'

# Cargar las credenciales desde el archivo JSON
credentials = service_account.Credentials.from_service_account_file(key_path)

# Crear un cliente de BigQuery utilizando las credenciales cargadas
client = bigquery.Client(credentials=credentials)


In [25]:
# Obtener la página HTML con los enlaces de los archivos Parquet
url = 'https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

# Obtener los enlaces de los archivos Parquet para los meses del año 2023
links = soup.find_all('a', href=True)
parquet_links = [link['href'] for link in links if 'yellow_tripdata_2023' in link['href'] or 'green_tripdata_2023' in link['href']]

# Cargar los datos de los archivos Parquet en BigQuery
client = bigquery.Client()
dataset_id = 'trip_records_clean'
table_id = 'allTrips_2013_2023_update'

job_config = bigquery.LoadJobConfig(
    autodetect=True,
    source_format=bigquery.SourceFormat.PARQUET,
)

for link in parquet_links:
    job_config.schema = [
        bigquery.SchemaField('pickup_datetime', 'TIMESTAMP'),
        bigquery.SchemaField('dropoff_datetime', 'TIMESTAMP'),
        bigquery.SchemaField('trip_distance', 'FLOAT64'),
        bigquery.SchemaField('payment_type', 'STRING'),
        bigquery.SchemaField('pickup_location_id', 'STRING'),
        bigquery.SchemaField('dropoff_location_id', 'STRING'),
        bigquery.SchemaField('total_amount', 'NUMERIC'),
        bigquery.SchemaField('tip_amount', 'NUMERIC'),
        bigquery.SchemaField('taxi_type', 'STRING'),
    ]

    uri = f'gs://{link}'
    load_job = client.load_table_from_uri(uri, dataset_id + '.' + table_id, job_config=job_config)
    load_job.result()


     trip_id vehicle_id     time_id           pickup_datetime  \
0    7261485       None  2013-01-16 2013-01-16 23:43:39+00:00   
1    7261500       None  2013-01-16 2013-01-16 23:43:45+00:00   
2    7261508       None  2013-01-16 2013-01-16 23:43:48+00:00   
3    7261510       None  2013-01-16 2013-01-16 23:43:48+00:00   
4    7261524       None  2013-01-16 2013-01-16 23:43:52+00:00   
..       ...        ...         ...                       ...   
995  7266453       None  2013-01-17 2013-01-17 00:01:00+00:00   
996  7266469       None  2013-01-17 2013-01-17 00:01:00+00:00   
997  7266472       None  2013-01-17 2013-01-17 00:01:00+00:00   
998  7266476       None  2013-01-17 2013-01-17 00:01:00+00:00   
999  7266487       None  2013-01-17 2013-01-17 00:01:00+00:00   

             dropoff_datetime trip_distance payment_type pickup_location_id  \
0   2013-01-17 00:02:28+00:00           8.9            2                 79   
1   2013-01-16 23:57:49+00:00           6.2            1     

In [26]:
df.head()

,trip_id,vehicle_id,time_id,pickup_datetime,dropoff_datetime,trip_distance,payment_type,pickup_location_id,dropoff_location_id,total_amount,tip_amount
0,7261485,None,2013-01-16,2013-01-16 23:43:39+00:00,2013-01-17 00:02:28+00:00,8.9,2,79,160,27.5,0
1,7261500,None,2013-01-16,2013-01-16 23:43:45+00:00,2013-01-16 23:57:49+00:00,6.2,1,166,249,25.2,4.2
2,7261508,None,2013-01-16,2013-01-16 23:43:48+00:00,2013-01-17 00:03:04+00:00,6,1,230,82,24.5,3
3,7261510,None,2013-01-16,2013-01-16 23:43:48+00:00,2013-01-17 00:01:04+00:00,9.9,1,229,95,42.25,8.45
4,7261524,None,2013-01-16,2013-01-16 23:43:52+00:00,2013-01-17 00:03:48+00:00,10.8,2,138,189,33.5,0
